In [12]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils.dummies import dummify_columns

In [14]:
data = pd.concat([
    pd.read_csv("../processed_data/bars.csv", index_col=0),
    pd.read_csv("../processed_data/myrealty.csv", index_col=0),
    pd.read_csv("../processed_data/bnakaran.csv", index_col=0)
], axis = 0)
data = data.reset_index(drop=True)

In [15]:
scraping_log = pd.read_csv("../scraping_results/scraping_log.csv", index_col=0)
rents = scraping_log[scraping_log.webpage.str.contains("rent")]
rents = rents[rents.success == True]
bnakaran = pd.read_csv("../scraping_results/data/bnakaran_apartments.csv", index_col=0)
myrealty = pd.read_csv("../scraping_results/data/myrealty_apartments.csv", index_col=0)

renting_ap = bnakaran[bnakaran.webpage.isin(rents.webpage)]
filtered_data = data[(data["source"] == 'bnakaran') & (data["id"].isin(renting_ap["id"]))]
data = data.drop(filtered_data.index)

renting_ap = myrealty[myrealty.webpage.isin(rents.webpage)]
renting_ap.id = renting_ap.id.astype(str)
filtered_data = data[(data["source"] == 'myrealty') & (data["id"].isin(renting_ap["id"]))]
data = data.drop(filtered_data.index)

/var/folders/67/47xpns1s0zn6h8g8npsy5_q80000gn/T/ipykernel_19600/167581633.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renting_ap.id = renting_ap.id.astype(str)


In [16]:
prices_full = data["price"] > 30000
data.loc[prices_full, "price"] = data.loc[prices_full, "price"] / data.loc[prices_full, "area"]

In [17]:
data = pd.get_dummies(data, columns = ["building_type", "condition"])

In [19]:
# data.to_csv("../processed_data/data_no_map.csv")
data.drop(columns = ["building_type_Other", "condition_Other"])

,source,id,price,area,rooms,floor,storeys,bathroom_count,new_building,F_Furniture,...,F_Internet,F_Elevator,latitude,longitude,building_type_Monolit,building_type_Panel,building_type_Stone,condition_Good Condition,condition_Needs Renovation/Repair,condition_New/Excellent Condition
0,bars,4-3-1430,2148.760331,60.5,2.0,3.0,13.0,1.0,1,0,...,0,0,44.503848,40.198558,0,0,0,0,1,0
1,bars,4-3-1430.1,2237.252862,96.1,3.0,4.0,13.0,2.0,1,0,...,0,0,44.503848,40.198558,0,0,0,0,1,0
2,bars,1-N-432,3580.562660,156.4,4.0,9.0,17.0,2.0,0,1,...,0,1,44.583871,40.216732,0,0,0,1,0,0
3,bars,1-N-429,5466.666667,75.0,3.0,3.0,12.0,2.0,0,0,...,0,1,44.504497,40.183219,0,0,0,1,0,0
4,bars,1-A-118,3043.478261,138.0,4.0,3.0,5.0,2.0,0,0,...,0,1,44.512284,40.195903,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,bnakaran,d117922,1923.076923,78.0,2.0,4.0,10.0,1.0,0,0,...,0,0,44.499482,40.180919,1,0,0,0,1,0
2798,bnakaran,d93008,2888.888889,90.0,3.0,3.0,5.0,2.0,0,0,...,0,0,44.484030,40.192541,0,0,1,1,0,0
2799,bnakaran,d117922,1923.076923,78.0,2.0,4.0,10.0,1.0,0,0,...,0,0,44.499482,40.180919,1,0,0,0,1,0
2800,bnakaran,d117893,3500.000000,180.0,4.0,3.0,9.0,2.0,1,0,...,0,0,44.514441,40.181718,1,0,0,1,0,0
